<a href="https://colab.research.google.com/github/19521242bao/CS231/blob/main/FInal_Project/CS231_FasterRCNN%2BRetina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install albumentations==0.4.6

     |████████████████████████████████| 117 kB 4.2 MB/s 
     |████████████████████████████████| 948 kB 27.5 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65172 sha256=230563a3f87b0a0d7dc17b662148f0bbfafe252d091ee3f989f3ca3cea24bb0c
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import json
import numpy as np
import cv2
import os
import re

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt

DIR_INPUT = '/content/drive/My Drive/CS231/za_traffic_2020/traffic_train'
DIR_TRAIN = f'{DIR_INPUT}/images'
DIR_TEST = f'/content/drive/My Drive/CS231/za_traffic_2020/traffic_public_test/images'

In [ ]:
%cd /content/drive/My Drive/CS231

/content/drive/My Drive/CS231


In [ ]:
!unzip za_traffic_2020.zip

Streaming output truncated to the last 5000 lines.
  inflating: za_traffic_2020/traffic_public_test/images/380.png  
  inflating: za_traffic_2020/traffic_public_test/images/388.png  
  inflating: za_traffic_2020/traffic_public_test/images/394.png  
  inflating: za_traffic_2020/traffic_public_test/images/398.png  
  inflating: za_traffic_2020/traffic_public_test/images/400.png  
  inflating: za_traffic_2020/traffic_public_test/images/405.png  
  inflating: za_traffic_2020/traffic_public_test/images/407.png  
  inflating: za_traffic_2020/traffic_public_test/images/414.png  
  inflating: za_traffic_2020/traffic_public_test/images/419.png  
  inflating: za_traffic_2020/traffic_public_test/images/420.png  
  inflating: za_traffic_2020/traffic_public_test/images/421.png  
  inflating: za_traffic_2020/traffic_public_test/images/423.png  
  inflating: za_traffic_2020/traffic_public_test/images/434.png  
  inflating: za_traffic_2020/traffic_public_test/images/451.png  
  inflating: za_traffic_2

In [ ]:
with open(f"{DIR_INPUT}/train_traffic_sign_dataset.json", "r") as fin:
    data = json.load(fin)

labels = []
bboxes = []
images_id = []
for i in range(len(data['annotations'])):
    labels.append(data['annotations'][i]['category_id'])
    bboxes.append(data['annotations'][i]['bbox'])
    images_id.append(data['annotations'][i]['image_id'])

train_df = pd.DataFrame()
train_df['labels'] = labels
train_df['bbox'] = bboxes
train_df['image_id'] = images_id
train_df.head()

,labels,bbox,image_id
0,2,"[880, 333, 19, 18]",3
1,3,"[1069, 355, 83, 83]",5
2,2,"[768, 480, 9, 7]",5
3,6,"[781, 337, 17, 15]",6
4,2,"[733, 352, 7, 8]",16
